In [3]:
%load_ext sql
%sql duckdb:///../data/referral.db --alias referralsDB

c:\Users\MatthewLyn\AppData\Local\Programs\Python\Python310\lib\site-packages\requests\__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.9) or chardet (5.0.0)/charset_normalizer (2.0.12) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "


In [4]:
%sql referralsDB

In [7]:
%sql CREATE OR REPLACE TABLE hcpcs as SELECT DISTINCT * FROM read_csv_auto('../data/reference/cpt_hcpcs.csv', header=True)

*  (referralsDB) duckdb:///../data/referral.db
Done.


Count
16991


In [3]:
%sql CREATE OR REPLACE TABLE icd10cm as SELECT DISTINCT * FROM read_csv('../data/reference/icd10cm.csv', header=True, columns={'code_value': 'VARCHAR', 'code_long_description': 'VARCHAR'})

*  (referralsDB) duckdb:///../data/referral.db
Done.


Count
95387


In [4]:
%%sql 
CREATE OR REPLACE TABLE cln_referrals as 

    SELECT DISTINCT * 
    FROM read_parquet('../data/clean_referrals/*.parquet')

*  (referralsDB) duckdb:///../data/referral.db
Done.


Count
55510


- No Auth Required ->	Appointment Scheduled or Member to Schedule or Unable to Contact
- Approved	       ->   Appointment Scheduled or Member to Schedule or Unable to Contact
- Denied	       ->   Open
- Withdrawn	       ->   Open

In [9]:
%%sql 
CREATE OR REPLACE TABLE bi_referrals as

    SELECT DISTINCT *

    , UPPER("Last Name") || UPPER("First Name") as patient_name

    , UPPER("Last Name") || UPPER("First Name") || "DOB" as patient_id

    , patient_id || coalesce("Plan ID", 'NA') || coalesce("Referring Provider NPI", 'NA') || "Referral Date" || coalesce("Procedure", 'NA') || coalesce(Specialty, 'NA') as Referral_keyid

    , CASE
        WHEN "Approval Status" IN ('No HP Auth Required', 'HP Approved', 'Approved (comments required)', 'Complete/no Auth# needed')  
            AND "Visit Status" IN ('Appointment Scheduled', 'Member to Schedule', 'Unable to Contact', 'Denied', 'Withdrawn')
            THEN 'Completed'
        WHEN "Approval Status" IN ('Denied- Per Insurance Plan', 'Denied- Per Medical Director Review')
            AND "Visit Status" = 'Open'
            THEN 'Completed'
        ELSE "Referral Status" END as "Updated Status"

    , RIGHT("Procedure", 5) as Proc_code
    , pc.code_short_description as Proc_name 

    , FORMAT(
        '({}) {}-{}'
        , substring("Mobile Phone", 1, 3), substring("Mobile Phone", 3, 3), substring("Mobile Phone", 6, 4)
    ) as "Fmt Mobile Phone"

    , FORMAT(
        '({}) {}-{}'
        , substring("Home Phone", 1, 3), substring("Home Phone", 3, 3), substring("Home Phone", 6, 4)
    ) as "Fmt Home Phone"

    , icd.code_long_description AS "Diagnosis Description"

    FROM cln_referrals AS rf

    LEFT JOIN icd10cm AS icd
        ON trim(rf."Diagnosis") = trim(icd.code_value)

    LEFT JOIN hcpcs AS pc
        on RIGHT("Procedure", 5) = pc.code_value

    WHERE "Visit Status" IS NOT NULL

*  (referralsDB) duckdb:///../data/referral.db
Done.


Count
55510


In [11]:
%%sql 
COPY (SELECT * FROM bi_referrals) TO '../data/report_data/2023-04-05.parquet' (FORMAT 'parquet'); 

*  (referralsDB) duckdb:///../data/referral.db


FloatProgress(value=0.0, layout=Layout(width='100%'), style=ProgressStyle(bar_color='black'))

Done.


Count
55510
